In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass
print("DONE")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

DONE


In [3]:
!python -V

Python 3.10.12


In [4]:
!python -V

Python 3.10.12


In [5]:
!pip install torch torchvision torchaudio pytorch-lightning numpy pandas scikit-learn scipy

In [ ]:
!pip install speechbrain

In [7]:
!pip install llvmlite==0.31.0
!pip install resampy

ERROR: Could not find a version that satisfies the requirement llvmlite==0.31.0 (from versions: none)
ERROR: No matching distribution found for llvmlite==0.31.0
ERROR: Could not find a version that satisfies the requirement resampy (from versions: none)
ERROR: No matching distribution found for resampy


In [8]:
!git clone https://github.com/jameslyons/python_speech_features
!python setup.py develop

Cloning into 'python_speech_features'...
fatal: unable to access 'https://github.com/jameslyons/python_speech_features/': Could not resolve host: github.com
python3: can't open file '/kaggle/working/setup.py': [Errno 2] No such file or directory


In [9]:
 !pip install python-speech-features

ERROR: Could not find a version that satisfies the requirement python-speech-features (from versions: none)
ERROR: No matching distribution found for python-speech-features


In [10]:
# Combined Speaker Recognition Script (Adapted for VoxCeleb)

# Dependencies
# pip install torch torchvision torchaudio pytorch-lightning numpy pandas scikit-learn scipy python-speech-features speechbrain

import os
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from speechbrain.processing.PLDA_LDA import PLDA, StatObject_SB, LDA, Ndx, fast_PLDA_scoring
from scipy.io import wavfile
from scipy.signal import fftconvolve
import resampy
from python_speech_features import mfcc

# Config Class
class Config:
    def __init__(self, batch_size=32, input_size=24, hidden_size=512, num_classes=None,
                 x_vector_size=512, x_vec_extract_layer=6, learning_rate=0.001, num_epochs=5):
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.x_vector_size = x_vector_size
        self.x_vec_extract_layer = x_vec_extract_layer
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs

# Dataset Class
class VoxCelebDataset(Dataset):
    def __init__(self, root_dir, augment=False, max_len=1000): # add max_len
        self.root_dir = root_dir
        self.augment = augment
        self.max_len = max_len # add max_len
        self.file_list, self.labels, self.label_map = self._load_files_and_labels()

    def _load_files_and_labels(self):
        file_list = []
        labels = []
        label_map = {}
        current_label = 0

        for speaker_id in sorted(os.listdir(self.root_dir)):
            speaker_path = os.path.join(self.root_dir, speaker_id)
            if os.path.isdir(speaker_path):
                if speaker_id not in label_map:
                    label_map[speaker_id] = current_label
                    current_label += 1
                for root, _, files in os.walk(speaker_path):
                    for file in files:
                        if file.endswith(".wav"):
                            file_list.append(os.path.join(root, file))
                            labels.append(label_map[speaker_id])
        return file_list, labels, label_map

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        filepath = self.file_list[idx]
        label = self.labels[idx]
        sample_rate, audio = wavfile.read(filepath)
        if self.augment:
            audio = self.augment_audio(audio)
        mfcc_features = mfcc(audio, samplerate=sample_rate, numcep=24)
        
        # Pad or truncate to max_len
        mfcc_features = self.pad_or_truncate(mfcc_features) # call padding function
        
        return torch.tensor(mfcc_features, dtype=torch.float32), label

    @staticmethod
    def augment_audio(audio):
        noise = np.random.normal(0, 0.1, audio.shape)
        return audio + noise

    def pad_or_truncate(self, features): # Padding function
        """Pads or truncates features to a fixed length."""
        if features.shape[0] < self.max_len:
            pad_width = ((0, self.max_len - features.shape[0]), (0, 0))
            features = np.pad(features, pad_width=pad_width, mode='constant')
        else:
            features = features[:self.max_len, :]
        return features

# TDNN Layer
class TdnnLayer(nn.Module):
    def __init__(self, input_size=24, output_size=512, context=[0], batch_norm=True, dropout_p=0.0):
        super().__init__()
        self.context = context
        self.kernel = nn.Conv1d(input_size, output_size, kernel_size=len(context))
        self.batch_norm = nn.BatchNorm1d(output_size) if batch_norm else None
        self.dropout = nn.Dropout(p=dropout_p) if dropout_p > 0 else None

    def forward(self, x):
        x = self.kernel(x.transpose(1, 2))
        if self.batch_norm:
            x = self.batch_norm(x)
        if self.dropout:
            x = self.dropout(x)
        return x.transpose(1, 2)

# Main Model
class SpeakerRecognitionModel(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.tdnn = TdnnLayer(config.input_size, config.hidden_size)
        self.fc = nn.Linear(config.hidden_size, config.num_classes)

    def forward(self, x):
        x = self.tdnn(x)
        x = x.mean(dim=1)
        return self.fc(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.config.learning_rate)

# PLDA Classifier
class PldaClassifier:
    @staticmethod
    def train(train_xv, train_label):
        stat_obj = StatObject_SB(train_xv, train_label)
        plda = PLDA()
        plda.fit(stat_obj)
        return plda

    @staticmethod
    def score(plda, test_xv, test_label):
        return fast_PLDA_scoring(plda, test_xv, test_label)

ModuleNotFoundError: No module named 'speechbrain'

In [ ]:
# Main Execution

root_dir = "./data/VoxCeleb"  # Replace with the path to your VoxCeleb dataset

# Dataset and DataLoader
dataset = VoxCelebDataset(root_dir, augment=True)
config = Config(num_classes=len(dataset.label_map))

dataloader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True)

# Model Training
model = SpeakerRecognitionModel(config)
trainer = pl.Trainer(max_epochs=config.num_epochs)
trainer.fit(model, dataloader)

print("Training Complete")